In [ ]:
%load_ext autoreload
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scqubits as scq
import plotly.graph_objects as go
from functools import partial

%autoreload 2
from zcu_tools.analysis.fluxdep import (
    VisualizeSpet,
    load_result,
    load_spects,
    flx2mA,
    mA2flx,
)
from zcu_tools.analysis.t1_curve import plot_eff_t1

In [ ]:
qub_name = "Q12_2D/Q4"

# Load data

## Parameters

In [ ]:
loadpath = f"../../result/{qub_name}/params.json"
_, params, mA_c, period, allows = load_result(loadpath)
EJ, EC, EL = params

mA_c = 4.46

print(allows)

if "r_f" in allows:
    r_f = allows["r_f"]

if "sample_f" in allows:
    sample_f = allows["sample_f"]


flxs = np.linspace(0.0, 1.0, 1000)
mAs = flx2mA(flxs, mA_c, period)

In [ ]:
fluxonium = scq.Fluxonium(*params, flux=0.5, cutoff=50, truncated_dim=6)
spectrumData = fluxonium.get_matelements_vs_paramvals(
    operator="n_operator", param_name="flux", param_vals=flxs, evals_count=40
)

## TwoTone Spectrum

In [ ]:
load_path = f"../../result/{qub_name}/spect.hdf5"

s_spects = load_spects(load_path)
s_spects.keys()

### Align cflx

In [ ]:
for spect in s_spects.values():
    shift = mA_c - spect["mA_c"]
    spect["mA_c"] += shift
    spect["spectrum"]["mAs"] += shift
    spect["points"]["mAs"] += shift

## Load Sample Points

In [ ]:
# loading points
loadpath = f"../../result/{qub_name}/freqs.csv"

freqs_df = pd.read_csv(loadpath)
s_mAs = freqs_df["Current (mA)"].values  # mA
s_fpts = freqs_df["Freq (MHz)"].values * 1e-3  # GHz
s_T1s = freqs_df["T1 (μs)"].values
s_T1errs = freqs_df["T1err (μs)"].values

# sort by flux
s_mAs, s_fpts, s_T1s, s_T1errs = tuple(
    np.array(a) for a in zip(*sorted(zip(s_mAs, s_fpts, s_T1s, s_T1errs)))
)


freqs_df.head(10)

In [ ]:
s_flxs = mA2flx(s_mAs, mA_c, period)

# Matrix elements

In [ ]:
show_idxs = [(i, j) for i in range(2) for j in range(5) if j > i]

fig = go.Figure()
for i, (from_, to_) in enumerate(show_idxs):
    fig.add_trace(
        go.Scatter(
            x=flxs,
            y=np.abs(spectrumData.matrixelem_table[:, from_, to_]),
            mode="lines",
            name=f"{from_}-{to_}",
            line=dict(width=2),
        )
    )
fig.update_layout(
    title=f"EJ/EC/EL = {EJ:.3f}/{EC:.3f}/{EL:.3f}",
    title_x=0.5,
    xaxis_title=r"$\phi_{ext}/\phi_0$",
    yaxis_title="Matrix elements",
)

fig.show()

In [ ]:
fig.write_html(f"../../result/{qub_name}/web/matrixelem.html", include_plotlyjs="cdn")
fig.write_image(f"../../result/{qub_name}/image/matrixelem.png", format="png")

# Plot Map

In [ ]:
vs = VisualizeSpet(s_spects, s_mAs, s_fpts, mAs, spectrumData.energy_table, allows)
vs.set_scatter_style(10, color=s_T1s)
fig = vs.create_figure()
fig.add_vline(x=mA_c * 1e3, line_dash="dash", line_color="black", name="Cavity freq")
fig.update_layout(title_text=f"EJ/EC/EL = {EJ:.3f}/{EC:.3f}/{EL:.3f}", showlegend=True)

fig.show()

## save to html

In [ ]:
fig.write_html(f"../../result/{qub_name}/web/map.html", include_plotlyjs="cdn")
fig.write_image(f"../../result/{qub_name}/image/map.png", format="png")

# T1 curve

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True, figsize=(8, 4))

ax1.errorbar(s_mAs, s_T1s, yerr=s_T1errs, fmt=".-", label="Current")
ax1.grid()
ax1.set_xlabel(r"Current (mA)")

ax1.set_ylabel(r"$T_1$ ($\mu s$)")
ax2 = ax1.secondary_xaxis(
    "top",
    functions=(
        partial(mA2flx, mA_c=mA_c, period=period),
        partial(flx2mA, mA_c=mA_c, period=period),
    ),
)
ax2.set_xlabel(r"$\phi_{ext}/\phi_0$")

fig.savefig(f"../../result/{qub_name}/image/T1s.png")
plt.show()

In [ ]:
Temp = 54.2e-3

In [ ]:
Q_cap = 3e5

fig, _ = plot_eff_t1(
    s_mAs,
    s_flxs,
    s_T1s,
    mA_c,
    period,
    fluxonium,
    name="Q_cap",
    noise_name="t1_capacitive",
    values=[Q_cap / 2, Q_cap, Q_cap * 2],
    Temp=Temp,
)


fig.savefig(f"../../result/{qub_name}/image/T1s_fit_Qcap.png")
plt.show()

In [ ]:
x_qp = 2e-6

fig, _ = plot_eff_t1(
    s_mAs,
    s_flxs,
    s_T1s,
    mA_c,
    period,
    fluxonium,
    name="x_qp",
    noise_name="t1_quasiparticle_tunneling",
    values=[x_qp / 2, x_qp, x_qp * 2],
    Temp=Temp,
)


fig.savefig(f"../../result/{qub_name}/image/T1s_fit_xqp.png")
plt.show()

In [ ]:
Q_ind = 1e6

fig, _ = plot_eff_t1(
    s_mAs,
    s_flxs,
    s_T1s,
    mA_c,
    period,
    fluxonium,
    name="Q_ind",
    noise_name="t1_inductive",
    values=[Q_ind / 2, Q_ind, Q_ind * 2],
    Temp=Temp,
)


fig.savefig(f"../../result/{qub_name}/image/T1s_fit_Q_ind.png")
plt.show()